In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

In [2]:
# Read file with Net Migration numbers with origin and destination
org_dest_csv = "Resources/UN_MigrantStockByOriginAndDestination_2017.csv"
# read a catalog of countries with their respective destinations
country_cat_csv = "Resources/Region_country_catalog.csv"

# Read data into Pandas data frames
org_dest_df = pd.read_csv(org_dest_csv)
country_cat_df = pd.read_csv(country_cat_csv)

In [3]:
# Columns with NO data come with "..", need to replace those with 0 
org_dest_df = org_dest_df.replace('..',0)

# Join migration data to the countries' catalog to get rid of all rows that DO NOT contain country information
# and to get region for destination countries
new_country_data = pd.merge(country_cat_df[["Country","Region"]],org_dest_df, on="Country")

In [4]:
# Use melt function to convert countries' columns into rows keeping values for "Country","Region" and "Year"

# Store the column names we want to melt in a list to be used in the melt operation
lst_ctry = new_country_data.columns[3:]

# melt columns into rows
origin_data_df = pd.melt(new_country_data,id_vars=["Country","Region","Year"], \
                    value_vars=lst_ctry, var_name='Origin', value_name='Tot Immigrants')

# Clean numbers of commas so they can be converted
origin_data_df.replace({'Tot Immigrants': ','}, {'Tot Immigrants': ''}, regex=True, inplace=True)
# Convert numbers to integers so they can be aggregated
origin_data_df[['Year','Tot Immigrants']] = origin_data_df[['Year','Tot Immigrants']].applymap(np.int32)

# Rename columns to make more sense and help with next merge
origin_data_df.columns = ["Destination","DRegion","Year","Country","Tot Immigrants"]

# merge with countries' catalog to get "Region", "Dev Level" and "Income Range" for countries of origin
origin_data_df = pd.merge(country_cat_df,origin_data_df, on="Country")
# rename the columns once we have added the region for the origination country
origin_data_df.columns = ["Origin","ORegion","Dev Level","Income Range","Destination","DRegion","Year","Tot Immigrants"]

# save the top 20 countries of origination for filtering later on (considering all 12 years of data)
# we are taking 21 countries and then dropping Syria since there is NO economic info on it
top20_ctrys_lst = origin_data_df.groupby("Origin").agg({'Tot Immigrants': 'sum'}).\
    sort_values('Tot Immigrants', ascending=False).head(21).index

# ----------
# #origin_data_df.nlargest(20, "Tot Immigrants")
# # origin_data_df.groupby("Country").agg({'Tot Immigrants': 'sum'}).\
# #     sort_values('Tot Immigrants', ascending=False).head(20)
# #origin_data_df.head()

In [30]:
# total number of origination migrants per country 
# #origin_data_df.nlargest(20, "Tot Immigrants")
# top20_df = origin_data_df.groupby(["Origin"]).agg({'Tot Immigrants': 'sum'}).\
#      sort_values('Tot Immigrants', ascending=False).head(21).reset_index()
# top20_df


# total number of origination migrants per region 
#origin_data_df.groupby("ORegion").agg({'Net Immigrants': 'sum'}).sort_values('Net Immigrants', ascending=False)

# Filter#origin_data_df.head()

,Origin,Tot Immigrants
0,India,55416860
1,Mexico,48741010
2,Russian Federation,41589224
3,China,35527020
4,Bangladesh,27254648
5,Ukraine,22810405
6,Pakistan,20804187
7,Philippines,19509515
8,Afghanistan,18497304
9,United Kingdom,18249798


In [6]:
# # Top 10 countries receiving people
#origin_data_df.groupby("Destination").agg({'Net Immigrants': 'sum'}).\
#    sort_values('Net Immigrants', ascending=False).head(10)
# # Show the destination region sorted by net number of migrants 
#origin_data_df.groupby("DRegion").agg({'Net Immigrants': 'sum'}).sort_values('Net Immigrants', ascending=False)

In [7]:
# BACKUP
# # Determine from the top 20, which are the countries they are mainly going to
# whereto_20_ctrys_df = top_20_ctrys_df.groupby(["DRegion","Destination","Origin"]).\
#     agg({'Tot Immigrants': 'sum'}).reset_index().\
#     groupby(["Origin","Destination"]).agg({'Tot Immigrants': 'sum'}).\
#     sort_values(['Origin','Tot Immigrants'], ascending=[True,False]).reset_index().groupby(["Origin"]).head(10)

In [34]:
# This will be used in charting
# Filter the dataframe to show top 20
top_20_ctrys_df = origin_data_df.loc[origin_data_df.Origin.isin(top20_ctrys_lst)]

# Determine from the top 20, which are the countries they are mainly going to
whereto_20_ctrys_df = top_20_ctrys_df.groupby(["Origin","DRegion","Destination"]).\
    agg({'Tot Immigrants': 'sum'}).sort_values(['Origin','Tot Immigrants'], ascending=[True,False]).\
    reset_index().groupby(["Origin"]).head(5)

whereto_20_ctrys_df.head()

,Origin,DRegion,Destination,Tot Immigrants
0,Afghanistan,ASIA,Iran (Islamic Republic of),9216348
1,Afghanistan,ASIA,Pakistan,6136811
2,Afghanistan,ASIA,Saudi Arabia,1317707
3,Afghanistan,EUROPE,Germany,340400
4,Afghanistan,NORTHERN AMERICA,United States of America,254773


In [9]:
# # Determine from the top 20, which are the 3 regions they are mainly going
# whereto_20_ctrys_df = top_20_ctrys_df.groupby(["DRegion","Destination","ORegion","Origin"]).\
#     agg({'Tot Immigrants': 'sum'}).sort_values(['Origin','Tot Immigrants'], ascending=[True,False]).reset_index().\
#     groupby(["Origin","DRegion"]).agg({'Tot Immigrants': 'sum'}).\
#     sort_values(['Origin','Tot Immigrants'], ascending=[True,False]).reset_index().groupby(["Origin"]).head(3)


In [10]:
# Pivot the years to be columns to help with plotting
top_20_ctrys_df = top_20_ctrys_df.groupby(["Origin","Dev Level","Income Range","Year"]).agg({'Tot Immigrants': 'sum'}). \
    sort_values('Tot Immigrants', ascending=False).unstack()

# Unstacking cause a multilevel, get rid of it
top_20_ctrys_df.columns = top_20_ctrys_df.columns.droplevel()

# Reset the index so country of origin become another column (to save the data)
top_20_ctrys_df = top_20_ctrys_df.reset_index()

In [11]:
top_20_ctrys_df

Year,Origin,Dev Level,Income Range,2005,2010,2015,2017
0,Afghanistan,Least developed,Low-income,3826255,4989209,4855376,4826464
1,Bangladesh,Least developed,Lower-middle-income,5765158,6742845,7246726,7499919
2,China,Less Developed,Upper-middle-income,7242306,8648435,9674720,9961559
3,Egypt,Less Developed,Lower-middle-income,1900048,2611995,3200613,3412957
4,Germany,More Developed,High-income,3585342,3849980,4032528,4207957
5,India,Less Developed,Lower-middle-income,9649626,13320745,15859419,16587070
6,Indonesia,Less Developed,Lower-middle-income,2714351,3486338,3974691,4233788
7,Italy,More Developed,High-income,2739747,2612391,2872011,3029145
8,Kazakhstan,Less Developed,Upper-middle-income,3718926,3811293,3905768,4074446
9,Mexico,Less Developed,Upper-middle-income,10816544,12413073,12546524,12964869


In [35]:
# # Write the new dataframe to a new CSV file
# top_20_ctrys_df.to_csv("Output/Clean_Top20_migrant.csv", index=False, header=True)
# whereto_20_ctrys_df.to_csv("Output/whereto_20_ctrys.csv", index=False, header=True)
whereto_20_ctrys_df.to_csv("Output/top20_ctrys.csv", index=False, header=True)